Otimização de hiperparâmetros com optuna --- exemplo guiado
===========================================================



Este notebook é um exemplo guiado do uso do módulo `optuna` para a otimização de hiperparâmetros de um modelo de aprendizado de máquina. O `optuna` é um módulo para resolver problemas de otimização com parâmetros numéricos e categóricos que  fornece buscas mais inteligentes do que a busca aleatória e mais eficientes do que a busca em grade.

Por se tratar de um exemplo guiado, esta será apenas uma apresentação breve do módulo. Para mais informações, veja as referências [1] e [2].



## Importações



Precisamos instalar o `optuna` caso ele já não esteja instalado.



In [1]:
try:
    import optuna

except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install optuna

In [2]:
import numpy as np
import pickle
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from optuna import create_study, Trial

## Constantes



In [3]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
NUM_FOLDS = 10

DATASET_NAME = "penguins"
FEATURES = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm"]
TARGET = ["body_mass_g"]

NUM_TENTATIVAS = 50

## Carregando os dados



In [4]:
df = sns.load_dataset(DATASET_NAME)

df = df.reindex(FEATURES + TARGET, axis=1)
df = df.dropna()

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

X = df.reindex(FEATURES, axis=1).values
y = df.reindex(TARGET, axis=1).values.ravel()

## Modelo e espaço de busca



A função `cria_instancia_modelo` serve para criar uma instância do modelo escolhido (no caso é uma floresta aleatória, mas observe que isto é fácil de alterar). Esta função recebe um objeto tipo &ldquo;tentativa&rdquo; do `optuna`. Aqui este objeto ficou com o nome em inglês (`trial`) para já nos acostumarmos com a terminologia da área.

Observe que o dicionário `parametros` dentro desta função tem como chaves os nomes dos argumentos do modelo (floresta aleatória no caso, veja a [documentação](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) em caso de dúvidas) e como valores os valores de cada argumento. Aqui está a &ldquo;mágica&rdquo; do `optuna`: os valores dos argumentos podem ser sorteados com as funções `trial.suggest_int` (para números inteiros), `trial.suggest_float` (para números reais) e `trial.suggest_categorical` (para dados categóricos). São com estas funções que delimitamos nosso *espaço de busca* dos hiperparâmetros do nosso modelo.

Observe que não somos obrigados a usar estas funções do `optuna`, as três últimas chaves do dicionário apresentam argumentos com valores fixos.



In [5]:
def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.
    """

    parametros = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 100),

        "criterion": trial.suggest_categorical("criterion", ["squared_error", "friedman_mse", "poisson"]),

        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20, log=True),

        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20, log=True),

        "max_features": trial.suggest_float("max_features_internal", 0, 1),

        "n_jobs": -1,

        "bootstrap": True,

        "random_state": SEMENTE_ALEATORIA,
    }

    model = RandomForestRegressor(**parametros)

    return model

## Função objetivo



A função objetivo de um problema de otimização é a função que irá computar a nossa métrica de interesse. Neste caso, a métrica de interesse é o RMSE médio obtido por validação cruzada.



In [6]:
def funcao_objetivo(
    trial,
    X,
    y,
    num_folds=NUM_FOLDS,
):
    """Função objetivo do optuna

    Referencia:
      https://medium.com/@walter_sperat/ using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451
    """

    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring="neg_root_mean_squared_error",
        cv=NUM_FOLDS,
    )

    # releia sobre scores no sklearn para relembrar porque tem um negativo abaixo
    return -metricas.mean()

## Otimização de hiperparâmetros



A otimização em si é realizada criando um objeto de estudo com o `create_study`. O argumento `direction` deve conter a string `"minimize"` caso seja um problema de minimização (nosso caso, queremos minimizar o RMSE) ou a string `"maximize"` caso seja um problema de maximização (por exemplo, maximizar a acurácia de um classificador).



In [7]:
objeto_de_estudo = create_study(direction="minimize")

[I 2023-11-07 15:16:03,022] A new study created in memory with name: no-name-5d603f3f-bf80-4c22-b918-58861cd5ed19


Para efetivamente rodar o otimizador precisamos de uma função objetivo que tenha apenas um argumento, o `trial`. Nossa `funcao_objetivo` definida acima não cumpre com este requisito. Vamos definir a `funcao_objetivo_parcial`.



In [8]:
def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino)

Finalmente, temos tudo para rodar nosso otimizador. Rodamos ele com o método `optimize`.



In [9]:
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2023-11-07 15:16:04,988] Trial 0 finished with value: 360.3792396221867 and parameters: {'n_estimators': 91, 'criterion': 'poisson', 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features_internal': 0.48168370476342537}. Best is trial 0 with value: 360.3792396221867.
[I 2023-11-07 15:16:06,467] Trial 1 finished with value: 363.14370441214385 and parameters: {'n_estimators': 74, 'criterion': 'squared_error', 'min_samples_split': 16, 'min_samples_leaf': 7, 'max_features_internal': 0.4479378153560567}. Best is trial 0 with value: 360.3792396221867.
[I 2023-11-07 15:16:07,653] Trial 2 finished with value: 364.6542854943984 and parameters: {'n_estimators': 56, 'criterion': 'poisson', 'min_samples_split': 8, 'min_samples_leaf': 14, 'max_features_internal': 0.8945010255937467}. Best is trial 0 with value: 360.3792396221867.
[I 2023-11-07 15:16:09,169] Trial 3 finished with value: 348.2722564256648 and parameters: {'n_estimators': 78, 'criterion': 'squared_error', 'min_samples_split':

## Observando resultados



Vamos observar os resultados.



In [10]:
df = objeto_de_estudo.trials_dataframe()

df

,number,value,datetime_start,datetime_complete,duration,params_criterion,params_max_features_internal,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,360.379240,2023-11-07 15:16:03.137303,2023-11-07 15:16:04.988058,0 days 00:00:01.850755,poisson,0.481684,6,5,91,COMPLETE
1,1,363.143704,2023-11-07 15:16:04.988058,2023-11-07 15:16:06.467578,0 days 00:00:01.479520,squared_error,0.447938,7,16,74,COMPLETE
2,2,364.654285,2023-11-07 15:16:06.473509,2023-11-07 15:16:07.653457,0 days 00:00:01.179948,poisson,0.894501,14,8,56,COMPLETE
3,3,348.272256,2023-11-07 15:16:07.660468,2023-11-07 15:16:09.169623,0 days 00:00:01.509155,squared_error,0.537156,1,4,78,COMPLETE
4,4,363.031552,2023-11-07 15:16:09.169623,2023-11-07 15:16:10.745519,0 days 00:00:01.575896,friedman_mse,0.088455,8,12,82,COMPLETE
5,5,390.118678,2023-11-07 15:16:10.745519,2023-11-07 15:16:11.655415,0 days 00:00:00.909896,friedman_mse,0.510590,16,13,35,COMPLETE
6,6,393.249961,2023-11-07 15:16:11.655415,2023-11-07 15:16:12.666933,0 days 00:00:01.011518,friedman_mse,0.130178,19,2,37,COMPLETE
7,7,356.874332,2023-11-07 15:16:12.674966,2023-11-07 15:16:13.540235,0 days 00:00:00.865269,friedman_mse,0.867665,1,2,22,COMPLETE
8,8,349.280425,2023-11-07 15:16:13.540235,2023-11-07 15:16:15.635698,0 days 00:00:02.095463,poisson,0.696674,5,2,93,COMPLETE
9,9,352.286429,2023-11-07 15:16:15.635698,2023-11-07 15:16:16.734319,0 days 00:00:01.098621,squared_error,0.254491,2,8,41,COMPLETE


Vamos checar qual foi o melhor resultado.



In [11]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 17
Parâmetros do melhor trial: {'n_estimators': 55, 'criterion': 'poisson', 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features_internal': 0.8204430003732026}


Finalmente, vamos criar um modelo com os melhores hiperparâmetros encontrados e checar a sua performance.



In [12]:
modelo = cria_instancia_modelo(melhor_trial)
modelo.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsao = modelo.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(RMSE)

301.9563378440249


## Referências



1.  Documentação do `optuna` [https://optuna.readthedocs.io/en/stable/index.html](https://optuna.readthedocs.io/en/stable/index.html)

2.  Tutoriais do `optuna` [https://optuna.readthedocs.io/en/stable/tutorial/index.html](https://optuna.readthedocs.io/en/stable/tutorial/index.html)

